In [1]:
import sys
import os
import pandas as pd

In [2]:
df = pd.read_csv('../Scripts/telegram_data.csv')

In [3]:
df

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1378,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:43+00:00,photos\@kuruwear_1378.jpg
1,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1377,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:35+00:00,photos\@kuruwear_1377.jpg
2,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1376,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:22+00:00,photos\@kuruwear_1376.jpg
3,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1375,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:04+00:00,photos\@kuruwear_1375.jpg
4,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1374,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-14 07:11:18+00:00,photos\@kuruwear_1374.jpg
...,...,...,...,...,...,...
2379,Et Online Shopping,@Etonline_shopping,15996,Lmn🪝Emporio Armani 🪝\n🪝 Noble Atmosphere \n🪝 L...,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15996.jpg
2380,Et Online Shopping,@Etonline_shopping,15995,👑ROLEX 🪄\n🦚 Noble Atmosphere \n🦚 Luxury Made \...,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15995.jpg
2381,Et Online Shopping,@Etonline_shopping,15994,NaN,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15994.jpg
2382,Et Online Shopping,@Etonline_shopping,15993,NaN,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15993.jpg


In [4]:
print(df.describe())

                 ID
count   2384.000000
mean   12549.365352
std     7175.656707
min        1.000000
25%     2260.000000
50%    16642.500000
75%    17493.250000
max    18750.000000


In [ ]:
print(df.info())